In [ ]:
import os,sys,re
import csv
import pandas as pd
import numpy as np
import codecs
import warnings
warnings.simplefilter('ignore')
from tqdm.autonotebook import tqdm
    
import json


import openpyxl


## データのロード

In [ ]:
#base-path
path = r"/workspace"
data_path = os.path.join(path,"data")

In [ ]:
with open(os.path.join(data_path,'output/ranking/WMD/cv_JDs_wmd.json'),"r") as f:
    JDs_wmd = json.load(f)
with open(os.path.join(data_path,'output/ranking/WMD/cv_DE_wmd.json'),"r") as f:
    DE_wmd = json.load(f)

with open(os.path.join(data_path,"output/marge_json/marge_cv.json"),"r") as f:
    cv = json.load(f)

with open(os.path.join(data_path,"output/marge_json/marge_JDs.json"),"r") as f:
    JDs = json.load(f)

with open(os.path.join(data_path,"output/marge_json/marge_DataEngineer.json"),"r") as f:
    DE = json.load(f)

In [ ]:
#wmdの値が小さい順にソート
sord_JDs_wmd = {}
for _key in JDs_wmd.keys():
    sord_JDs_wmd[_key] = sorted(JDs_wmd[_key].items(), key=lambda x:x[1])

In [ ]:
# ブックを作成
wb = openpyxl.Workbook()
wb.create_sheet(index=1, title='wmd_JDs_text')
sheet = wb['wmd_JDs_text']


#エクセルに書き込み
count = 0
#indexの書き込み
sheet["A1"] = "cv_keys"
sheet["A2"] = "cv_txts"
for num_cv,i in zip(sord_JDs_wmd.keys(),range(len(sord_JDs_wmd))):
    
    sheet[f"{chr(66+i)}1"] = num_cv #cvのキーをA列に書き込み
    sheet[f"{chr(66+i)}2"] =  cv[num_cv]["text"]#cvをB列に書き込み
    cols_num = 66+i
    #JDsを書き込み
    for n,value_list in enumerate(sord_JDs_wmd[num_cv][:10]):
        try:
            sheet[f"A{n+3}"] = f"top_{n+1}"
            sheet[f"{chr(66+i)}{n+3}"] =  JDs[value_list[0]]["text"]
        except:
            print(value_list[0])
            continue
    
    #下位10の求人をコピペ
    for j,value_list in enumerate(sord_JDs_wmd[num_cv][-9:]):
        try:
            sheet[f"A{n+4+j}"] = f"lower_{j+1}"
            sheet[f"{chr(66+i)}{n+4+j}"] =  JDs[value_list[0]]["text"]
        except:
            print(value_list[0])
            continue
wb.save(os.path.join(data_path,'output/ranking/WMD/eva_wmd.xlsx'))